Drawing a subset of data from biendata

In [3]:
import os, shutil, csv

In [11]:
original_dataset_dir = '/Users/mithyyin/Documents/GitHub/TeamEve/Classfication_small_datasets_inception_v3/waste_original_dataset' #directory name of your biendata
#original_dataset_dir =r'C:\Users\oscarscaro\Documents\GitHub\TeamEve\Classfication_small_datasets_inception_v3\images_withoutrect'
base_dir = './data_small' #create a directory for the data subset

#os.mkdir(base_dir)

#creating a new folder for each set
train_dir = os.path.join(base_dir, 'train') 
#os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation') 
#os.mkdir(validation_dir) 
test_dir = os.path.join(base_dir, 'test') 
#os.mkdir(test_dir)

In [12]:
#making a folder for each category inside train folder
#there are 204 categories with ID 1..204

for i in range(1,205):
    train_categories_dir = os.path.join(train_dir, str(i))
    #os.mkdir(train_categories_dir)
    
for i in range(1,205):
    validation_categories_dir = os.path.join(validation_dir, str(i))
    #os.mkdir(validation_categories_dir)
    
for i in range(1,205):
    test_categories_dir = os.path.join(test_dir,str(i))
    #os.mkdir(test_categories_dir)

In [13]:
#importing csv files, and change it into a list

with open('train.csv', newline='') as csvfile:
    data = list(csv.reader(csvfile))

In [14]:
data.pop(0)

['category_id', 'image_id', 'file_name']

In [15]:
print(data[i][1])
print(len(data))


11739
80000


In [16]:
#Testing on the theor
import os

cpt = sum([len(files) for r, d, files in os.walk("/Users/mithyyin/Documents/GitHub/TeamEve/Classfication_small_datasets_inception_v3/waste_original_dataset/2")])
print(cpt)
print(cpt/4 * 3/4)

400
75.0


In [17]:
#rewritten code fort iterating over a datasets
import os

#directory = os.fsencode('waste_original_dataset')

#for category in os.listdir(directory):
for i in range(1,205):
    count1 = 0
    file = os.path.join(original_dataset_dir, str(i))
    #fnames = ['cat.{}.jpg'.format(i) for i in data[i][1]]
    #training
    if i == 36:
        continue
    if i == 96:
        continue
    if i == 121:
        continue
    if i == 147:
        continue
    if i == 174:
        continue
    if i == 175: 
        continue
    for fname in os.listdir(file):
        cpt = sum([len(files) for r, d, files in os.walk(file)])   
        if count1 <= (3/4*cpt):
            src = os.path.join(file, fname)
            file_path = os.path.join(train_dir,str(i))
            dst = os.path.join(file_path, fname)
            shutil.copyfile(src, dst)
            count1 += 1
        if count1 > (3/4*cpt) and count1 < (cpt):
            src = os.path.join(file, fname)
            file_path = os.path.join(validation_dir,str(i))
            dst = os.path.join(file_path, fname)
            shutil.copyfile(src, dst)
            count1 += 1
    '''
        If count1 > 150 and count1<=200:
            src = os.path.join(file, fname)
            file_path = os.path.join(test_dir,str(i))
            dst = os.path.join(file_path, fname)
            shutil.copyfile(src, dst)
            count1 += 1
    '''

Training model building

In [33]:
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras.layers import Dense, Dropout, Activation, Flatten, Lambda, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.engine import Input, Model
from keras.optimizers import SGD
from keras.callbacks import Callback, LearningRateScheduler, ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
import keras.backend as K
import json
import time
import keras

In [ ]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, enable=True)

As of keras 2, the module keras.layers.merge doesn't have a generic public Merge-Layer. Instead you are supposed to import the subclasses like keras.layers.Add or keras.layers.Concatenate etc. directly (or their functional interfaces with the same names lowercase: keras.layers.add, keras.layers.concatenate etc.).

In [35]:
#complete building function of Resnet x Inception v2
# we reduce # filters by factor of 8 compared to original inception-v4
nb_filters_reduction_factor = 8

def inception_resnet_v2_stem(x):
    # in original inception-resnet-v2, conv stride is 2
    x = Convolution2D(32//nb_filters_reduction_factor, (3, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='valid', data_format="channels_last")(x)
    x = Convolution2D(32//nb_filters_reduction_factor, (3, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='valid', data_format="channels_last")(x)
    x = Convolution2D(64//nb_filters_reduction_factor, (3, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    
    # in original inception-resnet-v2, stride is 2
    a = MaxPooling2D((3, 3), strides=(1, 1), padding='valid', data_format="channels_last")(x)
    # in original inception-resnet-v2, conv stride is 2
    b = Convolution2D(96//nb_filters_reduction_factor, (3, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='valid', data_format="channels_last")(x)
   # x = merge([a, b], mode='concat', concat_axis=-1)
    x = keras.layers.concatenate([a,b],axis=-1)
    
    a = Convolution2D(64//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    a = Convolution2D(96//nb_filters_reduction_factor, (3, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='valid', data_format="channels_last")(a)
    b = Convolution2D(64//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    b = Convolution2D(64//nb_filters_reduction_factor, (7, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(b)
    b = Convolution2D(64//nb_filters_reduction_factor, (1, 7), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(b)
    b = Convolution2D(96//nb_filters_reduction_factor, (3, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='valid', data_format="channels_last")(b)
    x = keras.layers.concatenate([a,b],axis=-1)
    
    # in original inception-resnet-v2, conv stride should be 2
    a = Convolution2D(192//nb_filters_reduction_factor, (3, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='valid', data_format="channels_last")(x)
    # in original inception-resnet-v2, stride is 2
    b = MaxPooling2D((3, 3), strides=(1, 1), padding='valid', data_format="channels_last")(x)
    x = keras.layers.concatenate([a,b],axis=-1)
    
    x = Activation('relu')(x)
    return x

def inception_resnet_v2_A(x):
    shortcut = x
    
    a = Convolution2D(32//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    
    b = Convolution2D(32//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    b = Convolution2D(32//nb_filters_reduction_factor, (3, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(b)
    
    c = Convolution2D(32//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    c = Convolution2D(48//nb_filters_reduction_factor, (3, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(c)
    c = Convolution2D(64//nb_filters_reduction_factor, (3, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(c)
    
    x = keras.layers.concatenate([a,b,c],axis=-1)
    x = Convolution2D(384//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='linear',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    
    #x = merge([shortcut, x], mode='sum')
    x = keras.layers.add([shortcut,x])
    x = Activation('relu')(x)
    
    return x

def inception_resnet_v2_reduction_A(x):
    a = MaxPooling2D((3, 3), strides=(2, 2), padding='valid', data_format="channels_last")(x)
    b = Convolution2D(384//nb_filters_reduction_factor, (3, 3), strides=(2, 2), activation='relu',
                      kernel_initializer='he_normal', padding='valid', data_format="channels_last")(x)
    c = Convolution2D(256//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    c = Convolution2D(256//nb_filters_reduction_factor, (3, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(c)
    c = Convolution2D(384//nb_filters_reduction_factor, (3, 3), strides=(2, 2), activation='relu',
                      kernel_initializer='he_normal', padding='valid', data_format="channels_last")(c)
    
    x = keras.layers.concatenate([a,b,c],axis=-1)
    
    return x
    

def inception_resnet_v2_B(x):
    shortcut = x
    
    a = Convolution2D(192//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    
    b = Convolution2D(128//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    b = Convolution2D(160//nb_filters_reduction_factor, (1, 7), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(b)
    b = Convolution2D(192//nb_filters_reduction_factor, (7, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(b)
    
    x = keras.layers.concatenate([a,b],axis=-1)
    x = Convolution2D(1154//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='linear',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    
    #x = merge([shortcut, x], mode='sum')
    x = keras.layers.add([shortcut,x])
    x = Activation('relu')(x)
    
    return x

def inception_resnet_v2_reduction_B(x):
    a = MaxPooling2D((3, 3), strides=(2, 2), padding='valid', data_format="channels_last")(x)
    b = Convolution2D(256//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    b = Convolution2D(288//nb_filters_reduction_factor, (3, 3), strides=(2, 2), activation='relu',
                      kernel_initializer='he_normal', padding='valid', data_format="channels_last")(b)
    c = Convolution2D(256//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    c = Convolution2D(288//nb_filters_reduction_factor, (3, 3), strides=(2, 2), activation='relu',
                      kernel_initializer='he_normal', padding='valid', data_format="channels_last")(c)
    d = Convolution2D(256//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    d = Convolution2D(288//nb_filters_reduction_factor, (3, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(d)
    d = Convolution2D(320//nb_filters_reduction_factor, (3, 3), strides=(2, 2), activation='relu',
                      kernel_initializer='he_normal', padding='valid', data_format="channels_last")(d)
    
    x = keras.layers.concatenate([a,b,c,d],axis=-1)
    
    return x


def inception_resnet_v2_C(x):
    shortcut = x
    
    a = Convolution2D(192//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    
    b = Convolution2D(192//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    b = Convolution2D(224//nb_filters_reduction_factor, (1, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(b)
    b = Convolution2D(256//nb_filters_reduction_factor, (3, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(b)
    
    x = keras.layers.concatenate([a,b],axis=-1)
    x = Convolution2D(2048//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='linear',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    
   # x = merge([shortcut, x], mode='sum')
    x = keras.layers.add([shortcut,x])
    x = Activation('relu')(x)
    
    return x

In [36]:

img_rows, img_cols = 100, 100
img_channels = 3

nb_classes = 204

# in original inception-resnet-v2, these are 5, 10, 5, respectively
num_A_blocks = 1
num_B_blocks = 1
num_C_blocks = 1

inputs = Input(shape=(img_rows, img_cols, img_channels))

x = inception_resnet_v2_stem(inputs)
for i in range(num_A_blocks):
    x = inception_resnet_v2_A(x)
x = inception_resnet_v2_reduction_A(x)
for i in range(num_B_blocks):
    x = inception_resnet_v2_B(x)
x = inception_resnet_v2_reduction_B(x)
for i in range(num_C_blocks):
    x = inception_resnet_v2_C(x)

x = AveragePooling2D(pool_size=(4, 4), strides=(1, 1), padding='valid', data_format="channels_last")(x)
x = Dropout(0.5)(x)
x = Flatten()(x)

predictions = Dense(nb_classes, activation='softmax')(x)

model = Model(input=inputs, output=predictions)

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [37]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 98, 98, 4)    112         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 96, 96, 4)    148         conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 96, 96, 8)    296         conv2d_2[0][0]                   
____________________________________________________________________________________________

In [38]:
model.compile(optimizer='adam',  #optimizer=optimizers.RMSprop(lr=2e-5), could also try that
              loss='categorical_crossentropy',  
              metrics=['accuracy'])

In [19]:
#setting up all the parameter for training
batch_size = 16 #try 32, 128
epoch = 100 # try 50, 100. 
data_augmentation = True

# preprocessing the data

1 Read the picture files.

2 Decode the JPEG content to RGB grids of pixels.

3 Convert these into floating-point tensors.

4 Rescale the pixel values (between 0 and 255) to the [0, 1] interval (as you know, neural networks prefer to deal with small input values).

In [20]:
import os
base_dir = './data_small'
train_dir = os.path.join(base_dir, 'train') 
validation_dir = os.path.join(base_dir, 'validation') 
test_dir = os.path.join(base_dir, 'test') 

# WIth data augmentation

In [21]:

#preprocessing the data implementation
#using data implementation
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
rescale=1./255,
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True)

#changed here, major changes
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, #train_dir is the path where you store all the validaiton folder, chnage this
    target_size = (100,100), #try 1920,1080
    batch_size = batch_size,
    class_mode = 'categorical')

validation_generator = test_datagen.flow_from_directory( #debug here
    validation_dir, #train_dir is the path where you store all the validation folder, change this
    target_size = (100,100),
    batch_size = batch_size,
    class_mode = 'categorical')

Found 60132 images belonging to 204 classes.
Found 20858 images belonging to 204 classes.


# Without data agumentation

In [ ]:
#preprocessing the data implementation
#using data implementation
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)

#changed here, major changes
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, #train_dir is the path where you store all the validaiton folder, chnage this
    target_size = (100,100), #try 1920,1080
    batch_size = batch_size,
    class_mode = 'categorical')

validation_generator = test_datagen.flow_from_directory( #debug here
    validation_dir, #train_dir is the path where you store all the validation folder, change this
    target_size = (100,100),
    batch_size = batch_size,
    class_mode = 'categorical')

In [42]:
# saving the best model
# checkpoint
#from keras.callbacks import TensorBoard

filepath="waste_sort_weights_best_updated.h5"

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

#using tensorboard
callbacks_list = [keras.callbacks.TensorBoard(
    log_dir = 'my_log_dir',
    histogram_freq=0, #records activation histogram every 1 epoch
    embeddings_freq=0,
    write_graph=True,
    write_images=True#recoding embedding data every 1 epoch
                                             ),checkpoint]

#not using tensorboard
callbacks_list2 = [checkpoint]

In [43]:
#training the model, fit the data
history = model.fit_generator(
train_generator,
steps_per_epoch=100,
epochs=epoch,
validation_data=validation_generator,
validation_steps=50,
callbacks = callbacks_list2) #change here

Epoch 1/100
100/100 [==============================] - 38s 376ms/step - loss: 5.4648 - accuracy: 0.0088 - val_loss: 5.0904 - val_accuracy: 0.0075

Epoch 00001: val_accuracy improved from -inf to 0.00750, saving model to waste_sort_weights_best_updated.h5
Epoch 2/100
100/100 [==============================] - 34s 340ms/step - loss: 5.2040 - accuracy: 0.0069 - val_loss: 5.1480 - val_accuracy: 0.0075

Epoch 00002: val_accuracy did not improve from 0.00750
Epoch 3/100
100/100 [==============================] - 38s 382ms/step - loss: 5.1139 - accuracy: 0.0250 - val_loss: 5.4436 - val_accuracy: 0.0200

Epoch 00003: val_accuracy improved from 0.00750 to 0.02000, saving model to waste_sort_weights_best_updated.h5
Epoch 4/100
100/100 [==============================] - 38s 380ms/step - loss: 4.9932 - accuracy: 0.0269 - val_loss: 4.8464 - val_accuracy: 0.0350

Epoch 00004: val_accuracy improved from 0.02000 to 0.03500, saving model to waste_sort_weights_best_updated.h5
Epoch 5/100
100/100 [======


Epoch 00074: val_accuracy did not improve from 0.63750
Epoch 75/100
100/100 [==============================] - 55s 548ms/step - loss: 0.8305 - accuracy: 0.7563 - val_loss: 2.1536 - val_accuracy: 0.4937

Epoch 00075: val_accuracy did not improve from 0.63750
Epoch 76/100
100/100 [==============================] - 54s 535ms/step - loss: 0.8628 - accuracy: 0.7625 - val_loss: 2.2413 - val_accuracy: 0.4387

Epoch 00076: val_accuracy did not improve from 0.63750
Epoch 77/100
100/100 [==============================] - 53s 535ms/step - loss: 0.7473 - accuracy: 0.7812 - val_loss: 1.7260 - val_accuracy: 0.5750

Epoch 00077: val_accuracy did not improve from 0.63750
Epoch 78/100
100/100 [==============================] - 53s 535ms/step - loss: 0.7642 - accuracy: 0.7919 - val_loss: 0.7153 - val_accuracy: 0.5138

Epoch 00078: val_accuracy did not improve from 0.63750
Epoch 79/100
100/100 [==============================] - 53s 534ms/step - loss: 0.8054 - accuracy: 0.7656 - val_loss: 3.1861 - val_ac

In [66]:
#save the last epoch of the model 
model.save('wastesorting_resnet_inception_v2.h5') #model checkpoints for the last epoch

In [95]:
from keras.models import load_model
model = load_model('wastesorting_resnet_inception_v2.h5')

In [ ]:
#predicting the results

# from keras.models import load_model
# model = load_model('waste_sort_weights_best.h5')

In [44]:
#this is from my mac, we had differnt version.
#import the model_weights
from keras.models import load_model

model.load_weights('wastesorting_resnet_inception_v2.h5')

model.compile(optimizer='adam',  #optimizer=optimizers.RMSprop(lr=2e-5), could also try that
              loss='categorical_crossentropy',  
              metrics=['accuracy'])

# Real work down below, evaluating the model

In [51]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_dir = os.path.join(base_dir, 'test') 

#test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
test_dir,
target_size=(100,100),
batch_size=1,
class_mode='categorical',
shuffle=False)


Found 9899 images belonging to 204 classes.


In [52]:
test_loss,test_acc = model.evaluate_generator(test_generator)


In [53]:
print('test acc:',test_acc)
print('test loss:',test_loss)

test acc: 0.5791494250297546
test loss: 0.01624361239373684


# Running on new datasets

In [54]:
from keras.models import load_model
model = load_model('waste_sort_weights_best_updated.h5')

In [55]:
from keras.preprocessing.image import ImageDataGenerator

testdir = r'C:\Users\oscarscaro\Desktop\complete_test'


test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    directory=testdir,
    target_size=(100,100),
    batch_size=16,
    class_mode=None,
    shuffle=False)

Found 10000 images belonging to 1 classes.


In [59]:
import numpy as np

test_generator.reset()

pred = model.predict_generator(test_generator,verbose=1,steps=10000/16)
predicted_class_indices = np.argmax(pred,axis=1)

625/625 [==============================] - 185s 296ms/step


In [71]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [89]:
import pandas as pd

filenames=test_generator.filenames

print(filenames[0])
print(filenames[0][11:24])

filenames2 = [var[11:23] for var in filenames] 
#filenmaes = [filenames[k][12:23] for k in filenames]

results=pd.DataFrame({"Filename":filenames2,
                     "predictions":predictions})

test_image\0000381cb33e.jpg
0000381cb33e.


In [90]:
results

,Filename,predictions
0,0000381cb33e,92
1,000241c09e8e,9
2,000f0e3a30ea,202
3,002079c8b613,51
4,00217c35fd0c,142
5,002517f875f1,51
6,002c9b06e1f4,130
7,002df2b7289b,178
8,002ebc09417e,156
9,003486787257,26


In [91]:
#output, import it into csv.
export_csv = results.to_csv(r'C:\Users\oscarscaro\Desktop\Output\export.csv',index=None,header=True)

#compression_opts = dict(method='zip',
 #                      archive_name='out.csv')
#results.to_csv('test.zip',index=False,
 #            compression=compression_opts)